<a href="https://colab.research.google.com/github/kingtuler1454/torch/blob/main/5kab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.   Необходимо загрузить исходный набор данных и соответствующие метки классов.
2.   Произвести разделение загруженного набора данных на обучающую, тестовую и валидационую выборки (в соотношении 80:10:10). Проверить, что сформированные выборки сбалансированы.
3.   Написать модель нейронной сети для решения задачи классификации.
4.   Описать пайплайн предобработки данных. **ВАЖНО**: что так как ваш вариант предполагает работу с текстом, то необходимо выполниить векторизацию данных (подробности в туториале).
4.   Написать `train loop` (цикл обучения). Провести эксперименты по обучению с различными значениями параметров `learning rate` (скорость обучения) и `batch size` (размер мини-пакета). Выбрать по 3 значения для `learning rate` и `batch size` (итоговое количество экспериментов будет 9).
5.   Для каждого проведенного эксперимента вывести графики для значения функции потерь (ось `x` - итерация обучения/номер эпохи; ось `y` - значение функции потерь) и выбранной метрики качества (ось `x` - итерация обучения/номер эпохи; ось `y` - значение метрики качества). Графики необходимо выводить как для обучающей, так и для валидационной выборки.
6.   Оценить качество работы модели на тестовой выборке.
7.   Сделайте выводы по полученным результатам проведенных экспериментов. Какую модель из всех полученных стоит использовать?
8.   Сохранить обученную модель.
9.   Выполните повторную инициализацию модели и загрузку весов.  Продемонстрируйте работоспособность модели (пропустите через нее какой-то отзыв/рецензию и выведите результат).


https://www.kaggle.com/code/shivammehta007/spam-not-spam-classifier-with-pytorch

In [ ]:
import torch
import pandas as pd
from numpy import loadtxt


In [ ]:
torch.cuda.is_available()

True

In [ ]:
!nvidia-smi

Sun Dec 18 09:52:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Необходимо загрузить исходный набор данных и соответствующие метки классов.

In [37]:
    
with open("/content/number_star.txt","r+") as file:
  number_star=file.read()
  number_star=number_star.split(", ")

with open("/content/text_opinions.txt","r+") as file:
  text_opinion=file.read()
  text_opinion=text_opinion
  text_opinion=text_opinion.split("', '")


df = pd.DataFrame({"star": number_star, "text": text_opinion})
print(df)




     star                                               text
0       1  'Ужасный банк\n\nДостоинства: не наблюдается\n...
1       1  Испытываю ЧУВСТВО НЕНАВИСТИ еще с 90-х когда в...
2       1  Обман кредитных карт сбербанка\n\nДостоинства:...
3       1  Перестал работать сервис выставления счетов, т...
4       1  СМС кредитование, новый подход!\n\nДостоинства...
...   ...                                                ...
3789    5  Нормальный банк\n\nДостоинства: Доступно\nНедо...
3790    5  Отличный банк\n\nДостоинства: Удобный сервис, ...
3791    5  Вернули украденные с карты 27.12.2019 деньги\n...
3792    5  Отлично\n\nДостоинства: Оперативность\nНедоста...
3793  5\n  Хороший банк\n\nДостоинства: Удобно что через ...

[3794 rows x 2 columns]


Произвести разделение загруженного набора данных на обучающую, тестовую и валидационую выборки (в соотношении 80:10:10). Проверить, что сформированные выборки сбалансированы.

In [ ]:
0-3035 3035-379  3414-380